In [7]:
import pandas as pd

df = pd.read_csv("../data/SampleSuperstore.csv")
df.head()

df.info()
df.describe()
df.head(3), df.tail(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ship Mode     9994 non-null   object 
 1   Segment       9994 non-null   object 
 2   Country       9994 non-null   object 
 3   City          9994 non-null   object 
 4   State         9994 non-null   object 
 5   Postal Code   9994 non-null   int64  
 6   Region        9994 non-null   object 
 7   Category      9994 non-null   object 
 8   Sub-Category  9994 non-null   object 
 9   Sales         9994 non-null   float64
 10  Quantity      9994 non-null   int64  
 11  Discount      9994 non-null   float64
 12  Profit        9994 non-null   float64
dtypes: float64(3), int64(2), object(8)
memory usage: 1015.1+ KB


(      Ship Mode    Segment        Country         City       State  \
 0  Second Class   Consumer  United States    Henderson    Kentucky   
 1  Second Class   Consumer  United States    Henderson    Kentucky   
 2  Second Class  Corporate  United States  Los Angeles  California   
 
    Postal Code Region         Category Sub-Category   Sales  Quantity  \
 0        42420  South        Furniture    Bookcases  261.96         2   
 1        42420  South        Furniture       Chairs  731.94         3   
 2        90036   West  Office Supplies       Labels   14.62         2   
 
    Discount    Profit  
 0       0.0   41.9136  
 1       0.0  219.5820  
 2       0.0    6.8714  ,
            Ship Mode   Segment        Country         City       State  \
 9991  Standard Class  Consumer  United States   Costa Mesa  California   
 9992  Standard Class  Consumer  United States   Costa Mesa  California   
 9993    Second Class  Consumer  United States  Westminster  California   
 
       Postal

In [10]:
# make column names lowercase and replace spaces with underscores
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.columns

df.head(2)



,ship_mode,segment,country,city,state,postal_code,region,category,sub-category,sales,quantity,discount,profit
0,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,261.96,2,0.0,41.9136
1,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,731.94,3,0.0,219.5820


In [ ]:
#Dates were simulated to demonstrate Order-to-Cash analytics due to dataset limitations

import numpy as np

np.random.seed(42)

df['order_date'] = pd.to_datetime(
    np.random.choice(
        pd.date_range('2019-01-01', '2022-12-31'),
        size=len(df)
    )
)



In [14]:
df['ship_date'] = df['order_date'] + pd.to_timedelta(
    np.random.randint(1, 8, size=len(df)),
    unit='D'
)


In [15]:
df[['order_date', 'ship_date']].head()


,order_date,ship_date
0,2022-01-31,2022-02-04
1,2022-12-30,2023-01-05
2,2021-05-10,2021-05-14
3,2022-07-18,2022-07-23
4,2022-02-04,2022-02-07


In [17]:
df.isna().sum()


ship_mode       0
segment         0
country         0
city            0
state           0
postal_code     0
region          0
category        0
sub-category    0
sales           0
quantity        0
discount        0
profit          0
order_date      0
ship_date       0
dtype: int64

In [19]:
df = df.drop_duplicates()


In [21]:
df = df.reset_index(drop=True)
df['order_id'] = df.index + 1

df['customer_id'] = (
    df['segment'].astype(str) + "_" + df['city'].astype(str)
)

df[['customer_id', 'segment', 'city']].head()


,customer_id,segment,city
0,Consumer_Henderson,Consumer,Henderson
1,Consumer_Henderson,Consumer,Henderson
2,Corporate_Los Angeles,Corporate,Los Angeles
3,Consumer_Fort Lauderdale,Consumer,Fort Lauderdale
4,Consumer_Fort Lauderdale,Consumer,Fort Lauderdale


In [22]:
df['payment_due_date'] = df['order_date'] + pd.to_timedelta(30, unit='D')

df[['order_date', 'payment_due_date']].head()


,order_date,payment_due_date
0,2022-01-31,2022-03-02
1,2022-12-30,2023-01-29
2,2021-05-10,2021-06-09
3,2022-07-18,2022-08-17
4,2022-02-04,2022-03-06


In [23]:
np.random.seed(42)

df['payment_date'] = df['payment_due_date'] + pd.to_timedelta(
    np.random.randint(-5, 45, size=len(df)),
    unit='D'
)

df[['payment_due_date', 'payment_date']].head()


,payment_due_date,payment_date
0,2022-03-02,2022-04-04
1,2023-01-29,2023-02-21
2,2021-06-09,2021-06-18
3,2022-08-17,2022-09-23
4,2022-03-06,2022-03-08


In [25]:
df['payment_delay_days'] = (
    df['payment_date'] - df['payment_due_date']
).dt.days


In [26]:
df[['payment_due_date', 'payment_date', 'payment_delay_days']].head()


,payment_due_date,payment_date,payment_delay_days
0,2022-03-02,2022-04-04,33
1,2023-01-29,2023-02-21,23
2,2021-06-09,2021-06-18,9
3,2022-08-17,2022-09-23,37
4,2022-03-06,2022-03-08,2


In [28]:
df['payment_status'] = df['payment_delay_days'].apply(
    lambda x: 'Late' if x > 0 else 'On Time'
)

df[['payment_delay_days', 'payment_status']].head()


,payment_delay_days,payment_status
0,33,Late
1,23,Late
2,9,Late
3,37,Late
4,2,Late


In [29]:
df['payment_status'].value_counts()


payment_status
Late       8821
On Time    1173
Name: count, dtype: int64

In [30]:
def aging_bucket(days):
    if days <= 0:
        return 'On Time / Early'
    elif days <= 30:
        return '0–30 Days Late'
    elif days <= 60:
        return '31–60 Days Late'
    else:
        return '60+ Days Late'

df['aging_bucket'] = df['payment_delay_days'].apply(aging_bucket)

df[['payment_delay_days', 'aging_bucket']].head()


,payment_delay_days,aging_bucket
0,33,31–60 Days Late
1,23,0–30 Days Late
2,9,0–30 Days Late
3,37,31–60 Days Late
4,2,0–30 Days Late


In [31]:
df['aging_bucket'].value_counts()


aging_bucket
0–30 Days Late     6023
31–60 Days Late    2798
On Time / Early    1173
Name: count, dtype: int64

In [32]:
customer_risk = (
    df.groupby('customer_id')
      .agg(
          total_sales=('sales', 'sum'),
          avg_delay_days=('payment_delay_days', 'mean'),
          late_payment_count=('payment_status', lambda x: (x == 'Late').sum())
      )
      .reset_index()
)

customer_risk.head()


,customer_id,total_sales,avg_delay_days,late_payment_count
0,Consumer_Aberdeen,25.500,40.000000,1
1,Consumer_Abilene,1.392,0.000000,0
2,Consumer_Akron,885.188,18.769231,11
3,Consumer_Albuquerque,179.796,24.000000,4
4,Consumer_Alexandria,130.820,21.333333,3


In [33]:
customer_risk.sort_values(
    by=['late_payment_count', 'avg_delay_days'],
    ascending=False
).head(10)


,customer_id,total_sales,avg_delay_days,late_payment_count
261,Consumer_New York City,136869.8230,20.662698,451
211,Consumer_Los Angeles,89676.6405,20.094241,339
299,Consumer_Philadelphia,63203.2720,20.193309,233
631,Corporate_New York City,63383.2490,19.954887,231
352,Consumer_San Francisco,59195.0990,19.300000,228
590,Corporate_Los Angeles,46786.2385,18.417722,205
363,Consumer_Seattle,59016.3100,20.054795,194
162,Consumer_Houston,34046.0646,18.533019,183
58,Consumer_Chicago,25133.8030,19.586207,159
659,Corporate_Philadelphia,28890.8430,19.288136,154
